In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {
    "User-Agent": "Mozilla/5.0"
}

# Base URL for Tasty recipes (pagination example)
base_url = "https://tasty.co/topic/all?page={}"

def get_recipe_links(page):
    url = base_url.format(page)
    print(f"Fetching recipe links from: {url}")
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    links = []
    # Recipes are linked in article tags with <a href="/recipe/..."
    for a in soup.select("a.recipe-feed-item__link"):
        href = a.get("href")
        if href and href.startswith("/recipe/"):
            full_url = "https://tasty.co" + href
            links.append(full_url)
    return links

def scrape_recipe(url):
    print(f"Scraping recipe: {url}")
    try:
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")

        # Title
        title_tag = soup.find("h1", class_="recipe-name")
        title = title_tag.get_text(strip=True) if title_tag else "N/A"

        # Ingredients
        ingredients = [i.get_text(strip=True) for i in soup.select("li.ingredients__section-item")]

        # Instructions
        instructions = [step.get_text(strip=True) for step in soup.select("li.instructions__item")]

        return {
            "Title": title,
            "URL": url,
            "Ingredients": "; ".join(ingredients),
            "Instructions": " ".join(instructions)
        }
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")
        return None

all_recipes = []
pages_to_scrape = 5  # Change as needed (each page has ~10-12 recipes)

for page in range(1, pages_to_scrape + 1):
    recipe_links = get_recipe_links(page)
    print(f"Found {len(recipe_links)} recipes on page {page}")

    for link in recipe_links:
        data = scrape_recipe(link)
        if data:
            all_recipes.append(data)
        time.sleep(1)  # Be polite and avoid hammering the server

# Save to CSV
df = pd.DataFrame(all_recipes)
df.to_csv("tasty_recipes.csv", index=False, encoding="utf-8")
print(f"Saved {len(all_recipes)} recipes to tasty_recipes.csv")


Fetching recipe links from: https://tasty.co/topic/all?page=1
Found 0 recipes on page 1
Fetching recipe links from: https://tasty.co/topic/all?page=2
Found 0 recipes on page 2
Fetching recipe links from: https://tasty.co/topic/all?page=3
Found 0 recipes on page 3
Fetching recipe links from: https://tasty.co/topic/all?page=4
Found 0 recipes on page 4
Fetching recipe links from: https://tasty.co/topic/all?page=5
Found 0 recipes on page 5
Saved 0 recipes to tasty_recipes.csv


In [2]:
# Install the selenium library
!pip install selenium

# Install openpyxl for saving to .xlsx
!pip install openpyxl

# Install chromium-browser (needed for headless Chrome)
!sudo apt-get update
!sudo apt-get install -y chromium-browser

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd

options = Options()
# Configure Chrome options for headless execution with the correct path
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = '/usr/bin/chromium-browser' # Specify the path to chromium-browser

driver = webdriver.Chrome(options=options)

url = "https://www.eatthismuch.com/recipes/"
driver.get(url)
time.sleep(5)  # wait for page to load JS content

recipe_titles = []
recipe_links = []

# Find recipe cards
cards = driver.find_elements(By.CSS_SELECTOR, "a.recipe-card__link")

for card in cards:
    title = card.text
    link = card.get_attribute("href")
    recipe_titles.append(title)
    recipe_links.append(link)

driver.quit()

# Save to CSV
df = pd.DataFrame({"Title": recipe_titles, "URL": recipe_links})
df.to_csv("eatthismuch_recipes.csv", index=False)
print(f"Saved {len(recipe_titles)} recipes to eatthismuch_recipes.csv")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 27.7 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,387 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,969 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Hit:11 https://ppa.lau